In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

In [ ]:
#Observations
print("Heavier mice seem to have larger tumors, but the mouse weight could also be due to the larger tumors.")
print('For the mouse randomly selected on the Capomulin treatment, it seems the treatment did work, as the tumor size was reduce as time went on.')
print('The Ketapril treatment, the mouse tumors seemed to be much larger than all the other treatments, which could mean the treatment did not work or caused adverse effects.')
print('Overall, the Ramicane treatment seemed to work best as the tumor volumes were smallest out of all of the regiments tested on the mice.')

In [ ]:
# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

In [ ]:
# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

In [ ]:
# Combine the data into a single dataset
# Display the data table for preview
lab_rats = pd.merge(mouse_metadata, study_results, on="Mouse ID",suffixes = ("Metadata", "Study"))
lab_rats

In [ ]:
# Checking the number of mice.
len(lab_rats['Mouse ID'].value_counts())

In [ ]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
#find duplicate mouse ID values
dupes = lab_rats.drop_duplicates(subset=['Mouse ID','Timepoint'], keep = False)
dupes

In [ ]:
# Optional: Get all the data for the duplicate mouse ID. 

In [ ]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
mouse_IDs = dupes['Mouse ID'].unique()

In [ ]:
# Checking the number of mice in the clean DataFrame.
len(mouse_IDs)

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, 
#and SEM of the tumor volume for each regimen
# Use groupby and summary statistical methods to calculate the following properties 
#of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary dataframe.
tumor_data = lab_rats.groupby('Drug Regimen')['Tumor Volume (mm3)']
tumor_mean = tumor_data.mean()
tumor_median = tumor_data.median()
tumor_variance = tumor_data.var()
tumor_sd = tumor_data.std()
tumor_SEM = tumor_data.sem()
tumor_stats = pd.DataFrame({"Mean" : tumor_mean, "Median" : tumor_median,
                            "Variance" :tumor_variance,"Standard Deviation":tumor_sd,
                            "SEM" :tumor_SEM})
tumor_stats

In [ ]:
# Using the aggregation method, produce the same summary statistics in a single line
aggregation = tumor_data.aggregate(['mean', 'median','var','std', 'sem'])
aggregation

In [ ]:
# Generate a bar plot showing the total number of measurements taken on 
#each drug regimen using pandas.
measurements = dupes.drop_duplicates(subset=['Mouse ID'], keep = "last")
measurement_data = measurements.groupby('Drug Regimen')
measurement_data = measurement_data['Mouse ID'].count()
measurement_data.plot(kind="bar", title = "Mice per Drug Regimen")
plt.show()

In [ ]:
# Generate a bar plot showing the total number of measurements taken 
#on each drug regimen using pyplot.
drugs = measurements['Drug Regimen'].unique()

plt.bar(drugs, measurement_data,color = 'pink', width = .3)
plt.xticks(rotation=45)
plt.title("Mice per Drug Regimen")
plt.show()


In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pandas
f_v_m = dupes.drop_duplicates(subset=['Mouse ID'], keep = "last").groupby('Sex')
f_v_m = f_v_m['Mouse ID'].count()
f_v_m.plot(kind="pie",subplots = True, autopct='%1.1f%%')
plt.show()


In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot
plt.pie(f_v_m, labels = ("Female", "Male"),colors = ('Pink', 'Blue'), autopct='%1.1f%%')
plt.show()


In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse
last_timepoint = lab_rats.sort_values(by=['Timepoint'])
last_timepoint = last_timepoint.drop_duplicates(subset='Mouse ID', keep="last")

# Merge this group df with the original dataframe to get the tumor volume at the last timepoint
last_timepoint

In [ ]:
# Put treatments into a list for for loop (and later for plot labels)
#treatments = last_timepoint['Drug Regimen'].unique()
treatments = ("Capomulin", "Ramicane", "Infubinol", "Ceftamin")
# Create empty list to fill with tumor vol data (for plotting)


In [ ]:
# Locate the rows which contain mice on each drug and get the tumor volumes
tumor_data = {'Mouse ID': last_timepoint['Mouse ID'], 'Drug Regimen': last_timepoint['Drug Regimen'],\
              'Tumor Volume (mm3)' : last_timepoint['Tumor Volume (mm3)']}
tumor_df = pd.DataFrame(tumor_data)
tumor_df

In [ ]:
# Calculate the IQR and quantitatively determine if there are any potential outliers. 
# add subset     
box_data = last_timepoint.groupby('Drug Regimen')
box_plot = box_data['Tumor Volume (mm3)'].describe()
box_iqr = box_plot['75%'] - box_plot['25%']
box_plot['IQR'] = box_iqr
box_plot['Lower Bound'] = box_plot['25%'] - (1.5*box_plot['IQR'])
box_plot['Upper Bound'] = box_plot['75%'] + (1.5*box_plot['IQR'])
box_plot

In [ ]:
# Determine outliers using upper and lower bounds
outliers=[]

for treatment in treatments:
    upperbound = box_plot.loc[treatment,'Upper Bound']
    lowerbound = box_plot.loc[treatment,'Lower Bound']
    for drug in last_timepoint.itertuples():
        if drug[2] == treatment:
            #print(f'The drug {drug[2]} matches {treatment}.')
            if drug[7] > upperbound or drug[7] < lowerbound:
                #print(f' The tumor {drug[7]} is larger than {upperbound} or lower than {lowerbound}.')
                outliers.append(drug[1])
                
                
print(f' The outliers are mouse {outliers}.')

In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest
regimens = treatments
regimen_data = tumor_df[tumor_df['Drug Regimen'].isin(['Ketapril', 'Propriva', 'Zoniferol', 'Infubinol'])]
regimen_data

regimen_data.groupby('Drug Regimen').boxplot(subplots=False).set_xticklabels(regimens)
plt.ylabel('Final Tumor size in mm3')
plt.show()


In [ ]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin
mouse = 'u364'
mouse_data = lab_rats[lab_rats['Mouse ID'].isin([mouse])]
plt.plot(mouse_data['Timepoint'], mouse_data['Tumor Volume (mm3)'])
plt.xlabel('Timepoint')
plt.ylabel('Tumor Volume (mm3)')
plt.title('Tumor volume vs. Time point for a mouse treated with Capomulin')
plt.show()

In [ ]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen
Capomulin = last_timepoint[last_timepoint['Drug Regimen'].isin(['Capomulin'])]
plt.scatter(Capomulin['Weight (g)'], Capomulin['Tumor Volume (mm3)'],marker="+", facecolors = "green")
plt.xlabel('Weight (g)')
plt.ylabel('Tumor Volume (mm3)')
plt.title('Tumor volume vs. Average Weight for a mouse treated with Capomulin')
plt.show()

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
weight = Capomulin['Weight (g)']
tumor_vol = Capomulin['Tumor Volume (mm3)']
r = weight.corr(tumor_vol, method = 'pearson')
(slope, intercept, rvalue, pvalue, stderr) = st.linregress(weight, tumor_vol)
regress_values = weight * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(weight,tumor_vol)
plt.plot(weight,regress_values,"r-")
plt.annotate(line_eq,(21,25),fontsize=15,color="red")
plt.ylabel('Tumor Volume (mm3)')
plt.xlabel('Weight (g)')
plt.show()
print(f'Correlation coefficient is {round(r,2)}.')